In [32]:
VIDEO_FRAMERATE = 0.5 #nr of frames per second
#it takes my pc 3 min 15 s with framerate 0.5. For 25 fps, estimated is 2 h and 45 mins
PATH_TO_VIDEO = '2022-06-14_16-38-43_S04_eye-tracking-video-world_frame.mp4'
PATH_TO_INFO_AND_CROSS = "2022-06-14_16-38-43_streamLog_actionNet-wearables_S04.hdf5"
VIDEO_ID = "S04_01"
FRAMES_SAVE_PATH = "actionNet/" + VIDEO_ID
CALIBRATION_SHIFT = 14*60 + 21 # the first part of the video is usless

VIDEO_SAMPLING_RATE = 1/VIDEO_FRAMERATE

## Video transformation into frames

In [35]:
import os
os.makedirs(FRAMES_SAVE_PATH, exist_ok=True)

In [21]:
import cv2
def transform_to_frames(frameRate, path_to_video, frames_save_path, shift=0): 
    vidcap = cv2.VideoCapture(path_to_video)
    def getFrame(sec):
        vidcap.set(cv2.CAP_PROP_POS_MSEC,sec*1000)
        hasFrames,image = vidcap.read()
        if hasFrames:
            cv2.imwrite(frames_save_path+"/img_"+f'{count:010d}'+".jpg", image)     # save frame as JPG file
        return hasFrames
    sec = 0
    count=0
    success = getFrame(sec)
    while success:
        count = count + 1
        sec = sec + frameRate
        sec = round(sec, 2)
        success = getFrame(sec+shift)

In [ ]:
transform_to_frames(VIDEO_SAMPLING_RATE, PATH_TO_VIDEO, FRAMES_SAVE_PATH,CALIBRATION_SHIFT)

## Labels and dataframe (records)

In [25]:
import h5py, numpy as np


h5_file = h5py.File(PATH_TO_INFO_AND_CROSS)

device_name = 'experiment-activities'
stream_name = 'activities'

# Get the timestamped label data.
# As described in the HDF5 metadata, each row has entries for ['Activity', 'Start/Stop', 'Valid', 'Notes'].
activity_datas = h5_file[device_name][stream_name]['data']
activity_times_s = h5_file[device_name][stream_name]['time_s']
activity_times_s = np.squeeze(np.array(activity_times_s))  # squeeze (optional) converts from a list of single-element lists to a 1D list
# Convert to strings for convenience.
activity_datas = [[x.decode('utf-8') for x in datas] for datas in activity_datas]

# Combine start/stop rows to single activity entries with start/stop times.
#   Each row is either the start or stop of the label.
#   The notes and ratings fields are the same for the start/stop rows of the label, so only need to check one.
exclude_bad_labels = True # some activities may have been marked as 'Bad' or 'Maybe' by the experimenter; submitted notes with the activity typically give more information
activities_labels = []
activities_start_times_s = []
activities_end_times_s = []
activities_ratings = []
activities_notes = []
for (row_index, time_s) in enumerate(activity_times_s):
  label    = activity_datas[row_index][0]
  is_start = activity_datas[row_index][1] == 'Start'
  is_stop  = activity_datas[row_index][1] == 'Stop'
  rating   = activity_datas[row_index][2]
  notes    = activity_datas[row_index][3]
  if exclude_bad_labels and rating in ['Bad', 'Maybe']:
    continue
  # Record the start of a new activity.
  if is_start:
    activities_labels.append(label)
    activities_start_times_s.append(time_s)
    activities_ratings.append(rating)
    activities_notes.append(notes)
  # Record the end of the previous activity.
  if is_stop:
    activities_end_times_s.append(time_s)

In [26]:
import pandas as pd
action_net = pd.read_pickle("./action-net/ActionNet_train.pkl")
action_dict = {}
for i,row in action_net[["description", "labels"]].iterrows():
    desc = row[0]
    label = row[1]
    action_dict[desc] = label

In [27]:
labels_dict = {}
i=0
for label in set(action_net["labels"]):
    labels_dict[label] = i
    i+=1

{'Clean',
 'Clear',
 'Get/Put',
 'Load',
 'Open/Close',
 'Peel',
 'Pour',
 'Set',
 'Slice',
 'Spread',
 'Stack',
 'Unload'}

In [38]:
#TODO decide if we want to merge the same actions into one
#TODO reduce to only actions
from datetime import datetime
import pandas as pd

#seeding uuid 
import uuid
import random
rd = random.Random()
rd.seed(9341)

#"start_timestamp", "stop_timestamp"
records = []
shift = datetime.utcfromtimestamp(min(activities_start_times_s))
for i, label in enumerate(activities_labels) :
    #activities_end_times_s
    #start_timestamp = datetime.timestamp(datetime.utcfromtimestamp(activities_start_times_s[i]) - shift)
    #stop_timestamp = datetime.timestamp(datetime.utcfromtimestamp(activities_end_times_s[i]) - shift)
    start_frame = int((datetime.utcfromtimestamp(activities_start_times_s[i]) - shift).total_seconds() * VIDEO_FRAMERATE)
    stop_frame = int((datetime.utcfromtimestamp(activities_end_times_s[i]) - shift).total_seconds() * VIDEO_FRAMERATE)
    narration = label
    verb = action_dict[label]
    verb_class = labels_dict[verb]
    uuid_str = uuid.UUID(int=rd.getrandbits(128))
    records.append([uuid_str, VIDEO_ID, start_frame,stop_frame, narration, verb, verb_class])

records_pd = pd.DataFrame(records,columns=["uuid", "video_id","start_frame", "stop_frame", "narration", "verb", "verb_class"])

In [ ]:
records_pd

In [40]:
#TODO save as pkl